# Scales

In [25]:
# Let's bring in pandas as normal
import pandas as pd

# Here’s an example. Lets create a dataframe of letter grades in descending order. We can also set an index
# value and here we'll just make it some human judgement of how good a student was, like "excellent" or "good"

df=pd.DataFrame(['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D'],
                index=['excellent', 'excellent', 'excellent', 'good', 'good', 'good', 
                       'ok', 'ok', 'ok', 'poor', 'poor'],
               columns=["Grades"])
df

,Grades
excellent,A+
excellent,A
excellent,A-
good,B+
good,B
good,B-
ok,C+
ok,C
ok,C-
poor,D+


In [26]:
# Now, if we check the datatype of this column, we see that it's just an object, since we set string values
df.dtypes

Grades    object
dtype: object

In [27]:
# We can, however, tell pandas that we want to change the type to category, using the astype() function
df["Grades"].astype("category").head()

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
Name: Grades, dtype: category
Categories (11, object): [A, A+, A-, B, ..., C+, C-, D, D+]

In [28]:
# We see now that there are eleven categories, and pandas is aware of what those categories are. More
# interesting though is that our data isn't just categorical, but that it's ordered. That is, an A- comes
# after a B+, and B comes before a B+. We can tell pandas that the data is ordered by first creating a new
# categorical data type with the list of the categories (in order) and the ordered=True flag
my_categories=pd.CategoricalDtype(categories=['D', 'D+', 'C-', 'C', 'C+', 'B-', 'B', 'B+', 'A-', 'A', 'A+'], 
                           ordered=True)
# then we can just pass this to the astype() function
grades=df["Grades"].astype(my_categories)
grades.head()

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
Name: Grades, dtype: category
Categories (11, object): [D < D+ < C- < C ... B+ < A- < A < A+]

In [29]:
# Now we see that pandas is not only aware that there are 11 categories, but it is also aware of the order of
# those categoreies. So, what can you do with this? Well because there is an ordering this can help with
# comparisons and boolean masking. For instance, if we have a list of our grades and we compare them to a “C”
# we see that the lexicographical comparison returns results we were not intending. 

df[df["Grades"]>"C"]

,Grades
ok,C+
ok,C-
poor,D+
poor,D


In [30]:
# So a C+ is great than a C, but a C- and D certainly are not. However, if we broadcast over the dataframe
# which has the type set to an ordered categorical

grades[grades>"C"]

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
good         B-
ok           C+
Name: Grades, dtype: category
Categories (11, object): [D < D+ < C- < C ... B+ < A- < A < A+]

In [31]:
# We see that the operator works as we would expect. We can then use a certain set of mathematical operators,
# like minimum, maximum, etc., on the ordinal data.

In [32]:
# Sometimes it is useful to represent categorical values as each being a column with a true or a false as to
# whether the category applies. This is especially common in feature extraction, which is a topic in the data
# mining course. Variables with a boolean value are typically called dummy variables, and pandas has a built
# in function called get_dummies which will convert the values of a single column into multiple columns of
# zeros and ones indicating the presence of the dummy variable. I rarely use it, but when I do it's very
# handy.

In [33]:
# There’s one more common scale-based operation I’d like to talk about, and that’s on converting a scale from
# something that is on the interval or ratio scale, like a numeric grade, into one which is categorical. Now,
# this might seem a bit counter intuitive to you, since you are losing information about the value. But it’s
# commonly done in a couple of places. For instance, if you are visualizing the frequencies of categories,
# this can be an extremely useful approach, and histograms are regularly used with converted interval or ratio
# data. In addition, if you’re using a machine learning classification approach on data, you need to be using
# categorical data, so reducing dimensionality may be useful just to apply a given technique. Pandas has a
# function called cut which takes as an argument some array-like structure like a column of a dataframe or a
# series. It also takes a number of bins to be used, and all bins are kept at equal spacing.
 
# Lets go back to our census data for an example. We saw that we could group by state, then aggregate to get a
# list of the average county size by state. If we further apply cut to this with, say, ten bins, we can see
# the states listed as categoricals using the average county size.

# let's bring in numpy
import numpy as np

# Now we read in our dataset
df=pd.read_csv("datasets/census.csv")

# And we reduce this to country data
df=df[df['SUMLEV']==50]

# And for a few groups
df=df.set_index('STNAME').groupby(level=0)['CENSUS2010POP'].agg(np.average)

df.head()

STNAME
Alabama        71339.343284
Alaska         24490.724138
Arizona       426134.466667
Arkansas       38878.906667
California    642309.586207
Name: CENSUS2010POP, dtype: float64

In [34]:
# Now if we just want to make "bins" of each of these, we can use cut()
pd.cut(df,10)

STNAME
Alabama                   (11706.087, 75333.413]
Alaska                    (11706.087, 75333.413]
Arizona                 (390320.176, 453317.529]
Arkansas                  (11706.087, 75333.413]
California              (579312.234, 642309.586]
Colorado                 (75333.413, 138330.766]
Connecticut             (390320.176, 453317.529]
Delaware                (264325.471, 327322.823]
District of Columbia    (579312.234, 642309.586]
Florida                 (264325.471, 327322.823]
Georgia                   (11706.087, 75333.413]
Hawaii                  (264325.471, 327322.823]
Idaho                     (11706.087, 75333.413]
Illinois                 (75333.413, 138330.766]
Indiana                   (11706.087, 75333.413]
Iowa                      (11706.087, 75333.413]
Kansas                    (11706.087, 75333.413]
Kentucky                  (11706.087, 75333.413]
Louisiana                 (11706.087, 75333.413]
Maine                    (75333.413, 138330.766]
Maryland     

In [38]:
# Here we see that states like alabama and alaska fall into the same category, while california and the
# disctrict of columbia fall in a very different category.

# Now, cutting is just one way to build categories from your data, and there are many other methods. For
# instance, cut gives you interval data, where the spacing between each category is equal sized. But sometimes
# you want to form categories based on frequency – you want the number of items in each bin to the be the
# same, instead of the spacing between bins. It really depends on what the shape of your data is, and what
# you’re planning to do with it.

In [35]:
s = df.copy()
s

STNAME
Alabama                  71339.343284
Alaska                   24490.724138
Arizona                 426134.466667
Arkansas                 38878.906667
California              642309.586207
Colorado                 78581.187500
Connecticut             446762.125000
Delaware                299311.333333
District of Columbia    601723.000000
Florida                 280616.567164
Georgia                  60928.635220
Hawaii                  272060.200000
Idaho                    35626.863636
Illinois                125790.509804
Indiana                  70476.108696
Iowa                     30771.262626
Kansas                   27172.552381
Kentucky                 36161.391667
Louisiana                70833.937500
Maine                    83022.562500
Maryland                240564.666667
Massachusetts           467687.785714
Michigan                119080.000000
Minnesota                60964.655172
Mississippi              36186.548780
Missouri                 52077.626087
Monta

In [37]:
s = s.sort_values()
s

STNAME
South Dakota             12336.060606
North Dakota             12690.396226
Montana                  17668.125000
Nebraska                 19638.075269
Alaska                   24490.724138
Wyoming                  24505.478261
Kansas                   27172.552381
Iowa                     30771.262626
West Virginia            33690.800000
Idaho                    35626.863636
Kentucky                 36161.391667
Mississippi              36186.548780
Arkansas                 38878.906667
Vermont                  44695.785714
Oklahoma                 48718.844156
Missouri                 52077.626087
Virginia                 60111.293233
Georgia                  60928.635220
Minnesota                60964.655172
New Mexico               62399.363636
Tennessee                66801.105263
Indiana                  70476.108696
Louisiana                70833.937500
Alabama                  71339.343284
Colorado                 78581.187500
Wisconsin                78985.916667
Maine

In [40]:
cut_labels_6 = ['very low', 'low', 'average', 'high', 'very high']
cut_bins = [0, 100000, 300000, 400000, 500000, 700000]
pd.cut(s, bins=cut_bins)

STNAME
South Dakota                 (0, 100000]
North Dakota                 (0, 100000]
Montana                      (0, 100000]
Nebraska                     (0, 100000]
Alaska                       (0, 100000]
Wyoming                      (0, 100000]
Kansas                       (0, 100000]
Iowa                         (0, 100000]
West Virginia                (0, 100000]
Idaho                        (0, 100000]
Kentucky                     (0, 100000]
Mississippi                  (0, 100000]
Arkansas                     (0, 100000]
Vermont                      (0, 100000]
Oklahoma                     (0, 100000]
Missouri                     (0, 100000]
Virginia                     (0, 100000]
Georgia                      (0, 100000]
Minnesota                    (0, 100000]
New Mexico                   (0, 100000]
Tennessee                    (0, 100000]
Indiana                      (0, 100000]
Louisiana                    (0, 100000]
Alabama                      (0, 100000]
Colorado 